### Import modules

In [ ]:
import pandas as pd
import pickle
#from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import datetime
import matplotlib.animation as animation
import numpy as np

### Get noise data

In [ ]:
!scp shicks17@ese-kintamani.ese.ic.ac.uk:/home/shicks17/COVID_NOISE/BGS_noise.csv .
!scp shicks17@ese-kintamani.ese.ic.ac.uk:/home/shicks17/COVID_NOISE/*.pkl .

In [ ]:
df = pd.read_csv("BGS_noise.csv", index_col=0)
with open("params_BGS_IRIS.pkl", "rb") as f:  # Python 3: open(..., 'wb')
    datelist, station_list, dataset, nslc_all = pickle.load(f)
df.index=pd.to_datetime(df.index)
df['Date2'] = pd.to_datetime(df.index).date
df['weekday'] = pd.to_datetime(df.index)
df['weekday'] = (df['weekday'].dt.weekday)
df = df[df['Date2'] >= datetime.date(2020, 1 ,15)]
max_d = df["disp"].max()*1e9
min_d = 20
#df = df[df['weekday'] <= 4]   # Select only weekdays
pd.set_option('display.max_rows', None)
datelist = [d for d in datelist if d.weekday()<7 and d > datetime.date(2020,1,15)]
print(datelist)

### Get the Google mobility data

In [ ]:
df_goo = pd.read_csv("Global_Mobility_Report.csv")
df_goo = df_goo[df_goo["country_region_code"] == "GB"]
df_goo.index=pd.to_datetime(df_goo["date"])
df_goo = df_goo.groupby(df_goo.index.date).mean()
plt.plot(df_goo.index, df_goo["retail_and_recreation_percent_change_from_baseline"])

In [ ]:
# Get the vivago data

df_viv = pd.read_csv("vivacity_cars.csv")
df_viv.index=pd.to_datetime(df_viv["Date"], format='%d/%m/%Y')
print(df_viv)
plt.plot(df_viv.index, df_viv["Car"])

### Make the animation

In [ ]:
import matplotlib
plt.rcParams.update({'font.size': 11})

# Parameters to define
min_lat = 49
max_lat = 61
min_lon = -7
max_lon = 2
LOCKDOWN = datetime.datetime(2020, 3, 23)
LOCKDOWNe = datetime.datetime(2020, 5, 10)

LOCKDOWN2 = datetime.datetime(2020, 11, 2)
LOCKDOWN2e = datetime.datetime(2020, 12, 2)


# End of parameters to define

lines = []
lines2 = []
ll = []
def animate(i):
    """
    This is the beefy function to update the animation
    """
    print("Working on frame", i)
    # Remove previous plt.plot instances
    for line in lines:
        lines.remove(line)
        del line
    lines[:] = []
   
    # Remove axvline / axhlines objects apart from lockdown vertical line
    ax2.lines= [ax2.lines[0]]
    ax3.lines= []
    ax4.lines = [ax4.lines[0]]
    
    dl = (LOCKDOWN - datelist[i]).days
    df_filt = df[(df["Date2"] == datelist[i])]
    df_filt_rs = df_filt[(df_filt["net"]=="AM")]
    df_filt_gb = df_filt[(df_filt["net"]=="GB")]
    ax1.scatter(df_filt_gb["lon"], df_filt_gb["lat"], s=50, c=df_filt_gb["disp_pc"], zorder=10, **kw, label="BGS permanent station", marker="o")
    
    ax1.scatter(df_filt_rs["lon"], df_filt_rs["lat"], s=50, c=df_filt_rs["disp_pc"], zorder=10, **kw, label="RaspberryShake", marker="^")
    if dl >= 0:
        supstr = ("Seismic response in the UK to COVID-19 lockdown\n{:} : {:2.0f} days until lockdown 1.0"
                  .format(datelist[i].date(), dl))
    else:
        supstr = ("Seismic response in the UK to COVID-19 lockdown\n{:} : {:2.0f} days since lockdown 1.0"
                  .format(datelist[i].date(), dl*-1))
    plt.suptitle(supstr, fontsize=14)

    df_filt = df[(df["Date2"] <= datelist[i]) & (df["net"] == "AM")]
    means = df_filt.groupby(pd.Grouper(freq='1D')).median()
    lines.append(ax2.plot(means.index, means["disp"]*1e9, c="red", alpha=0.7,
                          label="RaspberryShake network average"))
    ax2.axvline(means.index[-1], c="k", alpha=0.5)
    ax2.axhline(means["disp"][-1]*1e9, c="red", alpha=0.5)

    df_filt = df[(df["Date2"] <= datelist[i]) & (df["net"] == "GB")]
    means = df_filt.groupby(pd.Grouper(freq='1D')).median()
    lines.append(ax3.plot(means.index, means["disp"]*1e9, c="blue", alpha=0.7,
                          label="BGS network average"))
    ax3.axhline(means["disp"][-1]*1e9, c="blue", alpha=0.5)
    lines_leg = ax2.get_lines() + ax3.get_lines()
    ax2.legend(lines_leg, [l.get_label() for l in lines_leg], loc=2)
    
    df_goo_filt = df_goo[(df_goo.index <= datelist[i])]
    lines.append(ax4.plot(df_goo_filt.index, df_goo_filt["retail_and_recreation_percent_change_from_baseline"],
                          color="m"))
    ax4.axvline(means.index[-1], c="k", alpha=0.5)
    if len(df_goo_filt) > 0:
        ax4.axhline(df_goo_filt["retail_and_recreation_percent_change_from_baseline"][-1], c="m", alpha=0.5)
    #df_viv_filt = df_viv[(df_viv.index <= datelist[i])]
    #lines.append(ax4.plot(df_viv_filt.index, df_viv_filt["Car"],
    #                      color="m"))
    #ax4.axvline(means.index[-1], c="k", alpha=0.5)
    #if len(df_viv_filt) > 0:
    #    ax4.axhline(df_viv_filt["Car"][-1], c="m", alpha=0.5)

# Make figure
fig = plt.figure(figsize=(20,12.6))
gs = fig.add_gridspec(4, 2)

# First, deal with the map subplot
ax1 = fig.add_subplot(gs[:, 0], projection=ccrs.PlateCarree())
ax1.set_extent([min_lon, max_lon, min_lat, max_lat], crs=ccrs.PlateCarree())
ax1.coastlines()
ax1.set_title("Noise level changes at individual seismic stations")
#m = Basemap(projection='merc',llcrnrlat=min_lat,urcrnrlat=max_lat,llcrnrlon=min_lon, urcrnrlon=max_lon,
#            resolution='i', ax=ax1)

#m.fillcontinents(color='tan',lake_color='lightblue')
#m.drawparallels(np.arange(min_lat,max_lat,1.),labels=[True,False,False,False],dashes=[2,2])
#m.drawmeridians(np.arange(min_lon,max_lon,1.),labels=[False,False,False,True],dashes=[2,2])
#m.drawmapboundary(fill_color='lightblue')
#m.drawcoastlines()

kw = dict(cmap=plt.cm.hot_r, vmin=-5, vmax=75, edgecolor="k")

# Initial dummy plots for making the map legend
df_filt_gb = df[(df["Date2"] == datelist[0]) & (df["net"] == "GB")]
sc = plt.scatter(df_filt_gb["lon"], df_filt_gb["lat"], c=df_filt_gb["disp_pc"], s=50, zorder=10, **kw, label="Brit. Geol. Surv. permanent station", marker="o")
df_filt_rs = df[(df["Date2"] == datelist[0]) & (df["net"] == "AM")]
ax1.scatter(df_filt_rs["lon"], df_filt_rs["lat"], c=df_filt_rs["disp_pc"], s=50, zorder=10, **kw, label="RaspberryShake citizen seismometer", marker="^")
cbar = plt.colorbar(sc)
cbar.set_label("% noise reduction (diplacement) relative to mean pre-lockdown weekday daytime noise")
plt.legend(loc=2)

# Now make the daily average network noise level subplot
ax2 = fig.add_subplot(gs[0:3, 1])
ax2.set_xlim([datetime.date(2020, 1 ,15), datetime.datetime(2020,12,20)])
l =ax2.axvline(LOCKDOWN, lw=3, ls="--", c="orange")
ax2.text(LOCKDOWN+datetime.timedelta(days=1), 15, "Gov't lockdown 1.0", fontsize=13, rotation=90,
         verticalalignment='bottom', ha="left", color="orange")
ll.append(l)

ax5 = ax2.twinx()
l = ax5.axvline(LOCKDOWN2, lw=3, ls="--", c="orange")
ax2.text(LOCKDOWN2+datetime.timedelta(days=1), 15, "Gov't lockdown 2.0", fontsize=13, rotation=90,
         verticalalignment='bottom', ha="left", color="orange")
ll.append(l)
ax5.axes.yaxis.set_visible(False)

ax6 = ax2.twinx()
l = ax6.axvline(LOCKDOWN2e, lw=3, ls="--", c="green")
ax6.text(LOCKDOWN2e+datetime.timedelta(days=1), 15, "End of Gov't lockdown 2.0", fontsize=13, rotation=90,
         verticalalignment='bottom', ha="left", color="green")
ll.append(l)
ax6.axes.yaxis.set_visible(False)

ax7 = ax2.twinx()
l = ax7.axvline(LOCKDOWNe, lw=3, ls="--", c="green")
ax7.text(LOCKDOWNe+datetime.timedelta(days=1), 15, "End of Gov't lockdown 1.0", fontsize=13, rotation=90,
         verticalalignment='bottom', ha="left", color="green")
ll.append(l)
ax7.axes.yaxis.set_visible(False)



ax2.set_ylim([0.1, 26])
ax2.set_title("Country-wide network average ground displacement")
ax2.set_ylabel("Mean daytime ground displacement in nanometers for RaspShake citizen seismometers", color="red")
ax2.tick_params(axis='y', colors='red')
fig.autofmt_xdate(bottom=0.2, rotation=30, ha='right')
ax3 = ax2.twinx()
ax3.set_ylabel("Ave. daytime ground displacement (nanometers) for BGS stations", color="blue")
ax3.set_ylim([0.1, 1.1])
ax3.tick_params(axis='y', colors='blue')

# Now make the Google mobility trend subplot
ax4 = fig.add_subplot(gs[3, 1],sharex=ax2)
ax4.set_title("Google mobility retail & recreation activity trends - UK wide average")
#ax4.set_title("Vivacity traffic activity - cars - UK wide average")
ax4.set_ylabel("% change in activity")
ax4.axvline(LOCKDOWN, lw=3, ls="--", c="orange")
ax14 = ax4.twinx()
ax14.axvline(LOCKDOWNe, lw=3, ls="--", c="green")
ax14.axvline(LOCKDOWN2, lw=3, ls="--", c="orange")
ax14.axvline(LOCKDOWN2e, lw=3, ls="--", c="green")
ax14.axes.yaxis.set_visible(False)

ax4.set_ylim([-100, 10])
#ax4.set_ylim([0.2, 1.1])
fig.autofmt_xdate(bottom=0.2, rotation=30, ha='right')

# Now make the animation
anim = animation.FuncAnimation(fig, animate, frames=len(datelist), interval=150,)
#plt.tight_layout(rect=[0, 0., 1, 0.96])
fig.subplots_adjust(left=0.05, bottom=0.06, right=0.95, top=0.85, wspace=None, hspace=None)
plt.text(x=0.5, y=0.94, s=
         "Animation created by Stephen Hicks, Imperial College London. Noise levels computed at frequencies of 5-15Hz.\n"
         "Contains British Geological Survey materials © UKRI [2020] and data from RaspberryShake citizen seismometers. "
         "Mobility data from Google.", 
         fontsize=12, ha="center", va="top", transform=fig.transFigure)
anim.save('UK_noise_animation.mp4', fps=6, extra_args=['-vcodec', 'libx264'], bitrate=10000)

In [ ]:
station_list
print(df.columns)

with open("params_BGS_IRIS.pkl", "rb") as f:  # Python 3: open(..., 'wb')
    datelist, station_list, dataset, nslc_all = pickle.load(f)

In [ ]:
import shapefile
from shapely.geometry import Point, shape
import geopandas as gpd

shp = gpd.read_file('/Users/sph1r17/Downloads/NUTS_Level_1__January_2018__Boundaries-shp/NUTS_Level_1__January_2018__Boundaries.shp')
shp = shp.to_crs(epsg=4326)
shp.to_file("result2.shp")

shp = shapefile.Reader('result2.shp')  
all_shapes = shp.shapes()
all_records = shp.records() 

all_shapes = shp.shapes()
all_records = shp.records() 


for dataset in ['BGS_IRIS', 'BGS_EIDA', 'RASPISHAKE']:
    with open("params_{}.pkl".format(dataset), "rb") as f:  # Python 3: open(..., 'wb')
        datelist, station_list, dataset, nslc_all = pickle.load(f)

    for station in station_list:
        df_sta = df[df["sta"] == station]
        df_sta = df_sta.sort_values(by=['Date2'])
        if len(df_sta) == 0:
            continue
        print(station)
        fig = plt.figure(figsize=(12, 6))
        plt.axvline(LOCKDOWN, ls='--', lw=1.2, label='Start of lockdown 1', c="red", zorder=20)
        plt.axvline(LOCKDOWNe, ls='--', lw=1.2, label='End of lockdown 1', c="green", zorder=20)


        net = df_sta['net'][0]
        point_to_check = (df_sta['lon'][0], df_sta['lat'][0])
        found = False

        for i in range(len(all_shapes)):
            boundary = all_shapes[i] # get a boundary polygon
            if Point(point_to_check).within(shape(boundary)): # make a point and see if it's in the polygon
                region = all_records[i][2] # get the second field of the corresponding record
                print(all_records[i])
                print("The point is in", region)
                if "England" in region or "Humber" in region or "London" in region:
                    region_short = 'england'
                    plt.axvline(LOCKDOWN2, ls='--', lw=1.2, label='Start of lockdown 2', c="red", zorder=20)
                    plt.axvline(LOCKDOWN2e, ls='--', lw=1.2, label='End of lockdown 2', c="green", zorder=20)
                    found = True
                elif "Scotland" in region:
                    region_short = 'scotland'
                    found = True
                elif "Wales" in region:
                    region_short = 'wales'
                    firebreak = datetime.datetime(2020, 10, 21)
                    firebreake = datetime.datetime(2020, 11, 23)
                    plt.axvline(firebreak, ls='--', lw=1.2, label='Start of firebreak', c="red", zorder=20)
                    plt.axvline(firebreake, ls='--', lw=1.2, label='End of firebreak', c="green", zorder=20)
                    found = True
                elif "Northern Ireland" in region:
                    region_short = 'n_ireland'
                    found = True
            if not found:
                region = "other"
                region_short = "other"
    
        plt.semilogy(df_sta['Date2'], df_sta['disp'] * 1e9, label="Daytime average displacement, 5-15 Hz")
        plt.title('{}.{}\n{}'.format(net, station, region))
        plt.grid(ls="--")


        plt.legend(loc="upper right")
        plt.xlabel("Date")
        plt.ylabel("Displacement (nm)")
        fig.autofmt_xdate()
        plt.savefig('uk-station-plots/{}/{}.{}.png'.format(region_short, net, station))